### ConversationTokenBufferMemory
* 대화의 히스토리를 메모리에 보관
* 대화의 개수가 아닌 토큰의 길이를 사용해 대화내용을 내보내줌

In [1]:
from dotenv import load_dotenv

load_dotenv()

True

In [17]:
from langchain_openai import ChatOpenAI
from langchain.memory import ConversationTokenBufferMemory 

llm = ChatOpenAI(
    temperature=0,
    model="gpt-4o"
)

memory = ConversationTokenBufferMemory(
    # 토큰수 계산을 위해, 실제 사용되는 LLM과 토큰화 방식을 일치시키기 위해
    llm=llm,
    max_token_limit=100,
    return_messages=True
)

In [18]:
memory.save_context(
    inputs={
        "human": "안녕하세요, 피자 한 판 주문하고 싶은데요. 어떤 피자가 있나요?"
    },
    outputs={
        "ai": "안녕하세요! 저희 매장의 인기 메뉴로는 페퍼로니, 치즈, 하와이안, 콤비네이션 피자가 있습니다. 어떤 피자를 선호하시나요?"
    },
)
memory.save_context(
    inputs={"human": "페퍼로니 피자로 할게요. 크기는 어떻게 되나요?"},
    outputs={
        "ai": "페퍼로니 피자는 M(25cm), L(32cm), XL(40cm) 사이즈로 제공됩니다. 크기를 선택해 주세요."
    },
)
memory.save_context(
    inputs={"human": "L 사이즈로 주문할게요. 도우는 어떤 종류가 있나요?"},
    outputs={
        "ai": "도우는 일반, 씬, 치즈크러스트 중에서 선택하실 수 있습니다. 어떤 도우로 하시겠어요?"
    },
)
memory.save_context(
    inputs={"human": "치즈크러스트로 할게요. 토핑 추가는 가능한가요?"},
    outputs={
        "ai": "네, 추가 토핑으로 모짜렐라 치즈, 올리브, 버섯, 피망, 양파를 선택하실 수 있습니다. 추가하실 토핑이 있으신가요?"
    },
)
memory.save_context(
    inputs={"human": "모짜렐라 치즈 추가로 주문할게요. 얼마인가요?"},
    outputs={
        "ai": "L사이즈 페퍼로니 피자(치즈크러스트) + 모짜렐라 치즈 추가는 총 28,000원입니다. 주문을 진행할까요?"
    },
)
memory.save_context(
    inputs={"human": "네, 주문할게요. 배달 시간은 얼마나 걸리나요?"},
    outputs={
        "ai": "주문이 완료되었습니다. 현재 예상 배달 시간은 약 40분입니다. 주문 현황은 문자메시지로 전송해 드리겠습니다. 감사합니다!"
    },
)

In [19]:
print(memory.load_memory_variables({})["history"])

[HumanMessage(content='네, 주문할게요. 배달 시간은 얼마나 걸리나요?', additional_kwargs={}, response_metadata={}), AIMessage(content='주문이 완료되었습니다. 현재 예상 배달 시간은 약 40분입니다. 주문 현황은 문자메시지로 전송해 드리겠습니다. 감사합니다!', additional_kwargs={}, response_metadata={})]


In [20]:
from langchain.chains import ConversationChain

conversation = ConversationChain(
    llm=llm,
    memory=memory
)

response = conversation.predict(
    input="주문한 내역을 알려주세요"
)

print(response)

주문 내역을 확인해 드리겠습니다. 고객님께서 주문하신 항목은 다음과 같습니다:

1. **마르게리타 피자** - 신선한 토마토 소스와 모짜렐라 치즈, 바질이 어우러진 클래식 피자입니다.
2. **치킨 윙** - 바삭하게 튀겨진 치킨 윙 6조각, 매콤한 소스와 함께 제공됩니다.
3. **시저 샐러드** - 신선한 로메인 상추와 크루통, 파마산 치즈가 들어간 샐러드로, 시저 드레싱이 곁들여져 있습니다.
4. **콜라 1.5L** - 시원한 콜라 한 병입니다.

주문하신 모든 항목은 약 40분 내에 배달될 예정입니다. 추가로 궁금한 사항이 있으시면 언제든지 말씀해 주세요!
